In [20]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image

In [21]:
# Пути
people_dir = 'all_images'  # <-- замени на путь к папке с фото людей

# Константы
IMG_SIZE = 300
BATCH_SIZE = 32
EPOCHS = 10


def load_images_from_folder(folder, label):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        try:
            img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
            img_array = image.img_to_array(img)
            images.append(img_array)
        except Exception as e:
            print(f"Ошибка при загрузке изображения {filename}: {e}. Пропускаем файл.")
            continue
    labels = np.full(len(images), label)  # создаём массив меток
    return np.array(images, dtype=np.float32), labels  # <-- ВАЖНО: возвращаем np.array!

# Загрузка фотографий людей
people_images, people_labels = load_images_from_folder(people_dir, label=1)

Ошибка при загрузке изображения .DS_Store: cannot identify image file <_io.BytesIO object at 0x14921d2b0>. Пропускаем файл.


In [22]:
people_images_2 = people_images[:300]
people_labels_2 = people_labels[:300]

In [23]:
# Генерация искусственных "пустых" изображений
num_fake = people_images_2.shape[0]
fake_images = np.random.rand(num_fake, IMG_SIZE, IMG_SIZE, 3).astype(np.float32) * 255.0  # Приводим к тому же диапазону

fake_labels = np.zeros(num_fake, dtype=np.int32)

In [24]:
# Склейка всех данных
all_images = np.concatenate([people_images_2, fake_images], axis=0) / 255.0  # Нормализация сразу после склейки
all_labels = np.concatenate([people_labels_2, fake_labels], axis=0)

In [25]:
# Перемешиваем данные
X_train, X_val, y_train, y_val = train_test_split(all_images, all_labels, test_size=0.2, random_state=42, stratify=all_labels)

In [26]:
# Создание tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1000).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH_SIZE)

In [27]:
# Построение модели
base_model = EfficientNetB3(
    weights='efficientnetb3_notop.h5',  # путь к локальному файлу
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

base_model.trainable = False

In [28]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)

In [29]:
model = Model(inputs=base_model.input, outputs=outputs)

In [30]:
# Компиляция
model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [31]:
# Обучение
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS
)

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 60s 4s/step - accuracy: 0.5139 - loss: 0.7079 - val_accuracy: 0.5000 - val_loss: 0.6574
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 48s 3s/step - accuracy: 0.5334 - loss: 0.7002 - val_accuracy: 0.5000 - val_loss: 0.7059
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 50s 3s/step - accuracy: 0.5630 - loss: 0.6716 - val_accuracy: 1.0000 - val_loss: 0.6085
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 49s 3s/step - accuracy: 0.7433 - loss: 0.6184 - val_accuracy: 1.0000 - val_loss: 0.5814
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 49s 3s/step - accuracy: 0.7616 - loss: 0.6149 - val_accuracy: 0.9917 - val_loss: 0.5557
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 50s 3s/step - accuracy: 0.7026 - loss: 0.6210 - val_accuracy: 1.0000 - val_loss: 0.5615
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 49s 3s/step - accuracy: 0.8042 - loss: 0.5937 - val_accuracy: 0.5000 - val_loss: 0.5732
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 49s 3s/step - accuracy: 0.7673 - loss: 0.5655 - val_accuracy: 1.0000 - val_loss:

In [32]:
# (Опционально) Разморозка части EfficientNetB3
base_model.trainable = True
fine_tune_at = 400

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [33]:
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS
)

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 59s 3s/step - accuracy: 0.8398 - loss: 0.4878 - val_accuracy: 1.0000 - val_loss: 0.4418
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 50s 3s/step - accuracy: 0.8427 - loss: 0.4897 - val_accuracy: 0.9917 - val_loss: 0.4901
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 48s 3s/step - accuracy: 0.8503 - loss: 0.4740 - val_accuracy: 1.0000 - val_loss: 0.4739
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 48s 3s/step - accuracy: 0.8494 - loss: 0.4698 - val_accuracy: 1.0000 - val_loss: 0.4461
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 49s 3s/step - accuracy: 0.8184 - loss: 0.5004 - val_accuracy: 1.0000 - val_loss: 0.4685
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 46s 3s/step - accuracy: 0.7941 - loss: 0.5077 - val_accuracy: 1.0000 - val_loss: 0.4670
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - accuracy: 0.8319 - loss: 0.4912 - val_accuracy: 1.0000 - val_loss: 0.4738
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - accuracy: 0.8364 - loss: 0.4697 - val_accuracy: 1.0000 - val_loss:

In [34]:
# Сохранение модели
model.save('efficientnetb3_people_detector_2.h5')

Тестирование

In [35]:
# Загрузка обученной модели
model = tf.keras.models.load_model('efficientnetb3_people_detector_2.h5')

# Путь к тестовым картинкам
test_dir = 'test'

# Функция для предсказания
def predict_image(img_path):
    img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # модель ожидает batch
    
    prediction = model.predict(img_array)[0][0]  # результат между 0 и 1
    if prediction > 0.5:
        print(f"{os.path.basename(img_path)}: Человек найден ({prediction:.2f})")
    else:
        print(f"{os.path.basename(img_path)}: Человека нет ({prediction:.2f})")

In [37]:
# Проверяем все тестовые картинки
for filename in os.listdir(test_dir):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(test_dir, filename)
        predict_image(img_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
IMG_8288.jpg: Человек найден (0.59)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
IMG_8289.jpg: Человек найден (0.72)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
8cf3b758d6af4abbaebc1b7bef7f4855.max-2500x1500.jpg: Человек найден (0.72)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
IMG_8290.jpg: Человек найден (0.73)
